# Парсинг сайта banki.ru

### Установка библиотек
Если вы не знаете установлены эти библиотеки или нет, то установите еще раз (второй раз установки ничего не сломает)

In [1]:
# pip install requests
# pip install pandas
# pip install time
# pip install ssl
# pip install bs4
# pip install openpyxl

### Импорты библиотек

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import time as t

### Код с опробированием методов

In [3]:
url = 'https://banki.ru/insurance/responses/company/sberbankstrahovaniezhizni'
r = requests.get(url)

In [4]:
r

<Response [200]>

In [25]:
r.text[:1000]

'<!doctype html>\n<html class="env-no-js env-desktop "\n\tprefix="ya: http://webmaster.yandex.ru/vocabularies/\nog: http://ogp.me/ns#" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml">\n\t<head >\n\t\t\t\t\t<link rel="shortcut icon" href="/static/favicons/favicon.ico">\n<link rel="icon" type="image/png" sizes="16x16" href="/static/favicons/favicon-16x16.png">\n<link rel="icon" type="image/png" sizes="32x32" href="/static/favicons/favicon-32x32.png">\n\n<meta name="mobile-web-app-capable" content="yes">\n<meta name="theme-color" content="#fff">\n<meta name="application-name" content="Banki.ru — финансовый супермаркет">\n<link rel="apple-touch-icon" sizes="57x57" href="/static/favicons/apple-touch-icon-57x57.png">\n<link rel="apple-touch-icon" sizes="60x60" href="/static/favicons/apple-touch-icon-60x60.png">\n<link rel="apple-touch-icon" sizes="72x72" href="/static/favicons/apple-touch-icon-72x72.png">\n<lin

In [27]:
soup = BeautifulSoup(r.text)
article = soup.find('article')
article

<article class="responses__item" data-id="3246" data-state="collapsed" data-test="responses-item" itemscope="" itemtype="http://data-vocabulary.org/Review">
<meta content="Сбербанк страхование" itemprop="itemreviewed"/>
<div class="flexbox flexbox--row flexbox--gap_xxsmall flexbox--align-items_flex-start">
<a class="header-h3" data-test="responses-header" href="/insurance/responses/company/response/3246/" itemprop="summary">Смещение сроков составления страхового акта, без предупреждения</a>
</div>
<div class="margin-top-xx-small color-gray-burn">
<div class="flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<span class="text-label" data-test="responses-estimate">Оценка:</span>
<span class="rating-grade rating-grade--small rating-grade--value-1" data-test="responses-rating-grade" itemprop="rating">
			1
		</span>
<span class="text-label" data-test="responses-status">
</span>
<

In [14]:
header = article.find('a', {'data-test': 'responses-header'}).text
header

'страхование жизни'

In [15]:
href = article.find('a', {'data-test': 'responses-header'}).get('href')
href

'/insurance/responses/company/response/28235/'

In [17]:
href = article.find('a', {'data-test': 'responses-header'}).get('href')
header = article.find('a', {'data-test': 'responses-header'}).text
text = article.find('div', {'itemprop': 'description'}).text.strip()
time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')

try:
    rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
except:
    rating = None
try:
    rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
except:
    rating_payouts = None
try:
    rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
except:
    rating_status = None

href, header, text, time, rating, rating_payouts, rating_status

('/insurance/responses/company/response/28235/',
 'страхование жизни',
 'ООО СК "Сбербанк"-организация-лохотрон!!!! Люди, опомнитесь, какое страхование,\xa0 заключив такого рода договор\xa0 вы содержите государство! в лице страховой компании Сбербанк! Никаких процентов, премий страховых выплачено\xa0 НЕ БУДЕТ!!!!! Это обман!Пенсионеры! у вас слишком много денег, не знаете как их потратить!? Живите в свое удовольствие! ни в коем случае не заключайте договоров со Сбербанком!',
 '2020-04-21 21:12:47',
 None,
 None,
 None)

In [28]:
df = pd.DataFrame(columns=[
    'url жалобы', 'Заголовок', 'Статус', 'Текст', 'Время', 'Оценка', 'Оценка выплат',
])

for article in soup.findAll('article'):
    try:
        href = article.find('a', {'data-test': 'responses-header'}).get('href')
        header = article.find('a', {'data-test': 'responses-header'}).text
        text = article.find('div', {'itemprop': 'description'}).text.strip()
        time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')

        # optional
        try:
            rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
        except:
            rating = None
        try:
            rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
        except:
            rating_payouts = None
        try:
            rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
        except:
            rating_status = None

        row = [href, header, rating_status, text, time, rating, rating_payouts]

        df.loc[len(df)] = row
    except:
        pass
    
df.head()

,url жалобы,Заголовок,Статус,Текст,Время,Оценка,Оценка выплат
0,/insurance/responses/company/response/3246/,"Смещение сроков составления страхового акта, б...",,День добрый.13.01.17г. у меня произошел страхо...,2017-02-20 09:39:51,1,None
1,/insurance/responses/company/response/3160/,Страхование без выплат,Оценка по выплатам:\n1,"Компанию ООО ""СБЕРБАНК СТРАХОВАНИЕ"" выбрал, т...",2017-02-08 13:51:55,None,1
2,/insurance/responses/company/response/3146/,"Игнорирование сотрудниками страховой, некомпет...",,"Добрый день.27 октября 2016 года, обратившись ...",2017-02-06 17:35:09,1,None
3,/insurance/responses/company/response/3116/,Не можем получить страховую выплату от СБЕРБАН...,,30 января 2017 года у меня Хатламаджиева А. М....,2017-02-02 11:56:43,1,None
4,/insurance/responses/company/response/3110/,Не возвращают страховую премию при расторжении...,Проблема решена,"Здравствуйте, но сложилась вопиющая ситуация, ...",2017-02-01 15:44:41,2,None


### Объединенный код 

In [12]:
ssl._create_default_https_context = ssl._create_unverified_context # чтобы Python соблюдал протокол https

base_url = 'https://banki.ru/'

companies = {
    'СБСЖ': {
        'pages': 50,  
        'url': 'insurance/responses/company/sberbankstrahovaniezhizni/',
    }, # СБСЖ 
    'СБС': {
        'pages': 17,
        'url': 'insurance/responses/company/sberbankstrahovanie/',
    }, # СБС 
}

page_url = '?page='

df = pd.DataFrame(columns=[
    'Компания', 'url жалобы', 'Заголовок', 'Статус', 'Текст', 'Время', 'Оценка', 'Оценка выплат',
])

for company in companies.keys():
    print(company)

    company_url = companies[company]['url']
    pages = companies[company]['pages']

    for page in range(1, pages + 1):
        t.sleep(1)
        
        r = requests.get(base_url + company_url + page_url + str(page))
        soup = BeautifulSoup(r.text)
        
        for article in soup.findAll('article'):
            try:
                href = article.find('a', {'data-test': 'responses-header'}).get('href')
                header = article.find('a', {'data-test': 'responses-header'}).text
                text = article.find('div', {'itemprop': 'description'}).text.strip()
                time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')
                
                # optional
                try:
                    rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
                except:
                    rating = None
                try:
                    rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
                except:
                    rating_payouts = None
                try:
                    rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
                except:
                    rating_status = None

                row = [company, base_url+href[1:], header, rating_status, text, time, rating, rating_payouts]

                df.loc[len(df)] = row
            except:
                pass

print(len(df))
df.to_excel('Отзывы_страхование.xlsx')

СБСЖ
СБС
1671


## Продолжение

In [56]:
df.to_excel('Отзывы_страхование(1).xlsx')

In [3]:
df = pd.read_excel('Отзывы_страхование(1).xlsx')

In [43]:
del df['Unnamed: 0']

In [125]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem()
russian_stopwords = stopwords.words("russian")

# Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text


df['preprocess_Текст'] = df['Заголовок'].apply(preprocess_text) + ' ' + df['Текст'].apply(preprocess_text)

[x] вид СС (смерть, кража, затопление)     
[x] дата написания отзыва
[x] тип обращения  
[ ] номер обращения в срм (16 цифр)  
[ ] вид продукта  
[ ] сумма  
[x] номер полиса  
[ ] ИНАЧЕ прочерк

In [5]:
type_ss = 'Вид_СС'
type_error = 'Тип обращения'
type_prod = 'Вид_продукта'
type_polis = 'Номер_полиса'
type_crm = 'номер обращения CRM'

In [51]:
df[type_ss] = None
# df[type_error] = None
df[type_error] = ''
df[type_prod] = None
df[type_polis] = None
df[type_crm] = None

In [52]:
# Вид страхового случая
dict_ss = {
    'смерт': 'смерть',
    'кража': 'кража',
    'украсть': 'кража',
    'затопление': 'затопление',
    'умирать': 'смерть',
    'уходить жизнь': 'смерть',
}
for find_ss, insert_ss in dict_ss.items():
    df.loc[df['preprocess_Текст'].str.contains(find_ss), type_ss] = insert_ss


# Тип обращения
dict_error = {
    'расторжение': 'расторжение',
    'ошибка': 'ошибка',
    'правлять дважды': 'повторный запрос документов',
    'запрашивать': 'повторный запрос документов',
    'изменение': 'внесение изменений в договоре',
    'возврат': 'возврат средств',
    'отказывать выплата': 'отказ в выплате',
    'отказ выплата': 'отказ в выплате',
    'давление': 'давление сотрудника',
    'внятный ответ': 'сотрудники не дают внятного ответа'
}
for find_error, insert_error in dict_error.items():
#     df.loc[(df['preprocess_Текст'].str.contains(find_error)) & (df[type_error].isnull()), type_error] = insert_error
    df.loc[(df['preprocess_Текст'].str.contains(find_error)), type_error] += insert_error + ' | '
df.loc[(df['Текст'].str.contains("не спрос")), type_error] = '| не спрашивают о страховке'
df.loc[(df['Текст'].str.contains("[^и] наступлени[еи] страхового случая")), type_error] = '| не выплачивают по сс'

# Вид продукта
dict_prod = {
    'билет в будущее': 'билет в будущее',
    'смарт': 'смарт',
    'защита дома': 'защита дома',
    'змдкр': 'ЗМДКР',
    'защищенный заемщик': 'защищенный заемщик',
    'защита близких плюс': 'защита близких плюс',
    'как зарплата': 'как зарплата',
    'защита квартиры': 'защита квартиры',
    'наследие': 'наследие',
    'семейный актив': 'семейный актив',
    'ВМСР50': 'смарт',
    'ЗМСРР101': 'ЗМСРР101',
    'будущий капитал': 'будущий капитал',
}
for find_prod, insert_prod in dict_prod.items():
    df.loc[df['Текст'].str.lower().str.replace('ё', 'е').str.contains(find_prod), type_prod] = insert_prod

    
# Номер полиса
for n_digits in ['10', '11', '12', '14']:
    df_prob = df['Текст'].str.extract('\D(\d{'+n_digits+'})\D')
    df.loc[df_prob[0].notnull(), type_polis] = "'" + df_prob[0]
    
    
# Номер обращения CRM
df_prob = df['Текст'].str.extract("(\d{6}[- ]\d{4}[- ]\d+)")
df.loc[df_prob[0].notnull(), type_crm] = df_prob[0]

In [53]:
for insert_error in dict_error.values():
    print(insert_error, '=', len(df[df[type_error].str.contains(insert_error)]))

расторжение = 506
ошибка = 97
повторный запрос документов = 70
повторный запрос документов = 70
внесение изменений в договоре = 51
возврат средств = 508
отказ в выплате = 51
отказ в выплате = 51
давление сотрудника = 10
сотрудники не дают внятного ответа = 25


In [7]:
df[(df[type_error] == '') & (df['Оценка'] < 4)]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Компания,url жалобы,Заголовок,Статус,Текст,Время,Оценка,Оценка выплат,preprocess_Текст,Вид_СС,Тип обращения,Вид_продукта,Номер_полиса,номер обращения CRM
0,0,0,0,СБСЖ,https://banki.ru/insurance/responses/company/r...,Страховой случай -смерть застрахованного. Тяну...,Проверяется,"Никогда бы не подумала, что организация с «гро...",2020-05-17 10:35:43,1.0,1.0,страховой случай смерть застраховывать тянуть ...,смерть,NaN,смарт,NaN,NaN
3,3,3,3,СБСЖ,https://banki.ru/insurance/responses/company/r...,Выплаты в Сбербанк страхование,Проверяется,Хотела бы поделиться своим опытом взаимодейств...,2020-05-15 15:25:58,1.0,2.0,выплата сбербанк страхование хотеть поделиться...,NaN,NaN,NaN,NaN,NaN
5,5,5,5,СБСЖ,https://banki.ru/insurance/responses/company/r...,Не пришел полис страхования жизни на почту!,Проверяется,Сегодня 14.05.2020 мною был оформлен полис стр...,2020-05-14 14:06:58,2.0,NaN,приходить полис страхование жизнь почта сегодн...,NaN,NaN,защищенный заемщик,NaN,NaN
8,8,8,8,СБСЖ,https://banki.ru/insurance/responses/company/r...,Сбербанк страхование жизни и забота о клиентах...,Проверяется,Уважаемая Мария Александрова так и не получили...,2020-05-12 19:39:30,1.0,2.0,сбербанк страхование жизнь забота клиент часть...,NaN,NaN,NaN,NaN,NaN
12,12,12,12,СБСЖ,https://banki.ru/insurance/responses/company/r...,Сбербанк страхование жизни и забота о клиентах,Проверяется,Уважаемые руководители ООО СК Сбербанк страхов...,2020-04-28 10:40:19,1.0,2.0,сбербанк страхование жизнь забота клиент уважа...,NaN,NaN,смарт,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,1658,1658,1658,СБС,https://banki.ru/insurance/responses/company/r...,Сроки рассмотрения заявлений о страховых выпла...,Оценка по выплатам:\n1,Всё красиво и просто на рекламных буклетах и б...,2016-08-02 11:23:23,1.0,1.0,срок рассмотрение заявление страховой выплата ...,NaN,NaN,защита дома,NaN,NaN
1659,1659,1659,1659,СБС,https://banki.ru/insurance/responses/company/r...,Осторожно! Сбербанк-Страхование!,NaN,"Хочу поделиться своей невеселой историей, нача...",2016-07-18 21:20:31,1.0,NaN,осторожно сбербанк страхование хотеть поделить...,NaN,NaN,NaN,NaN,NaN
1660,1660,1660,1660,СБС,https://banki.ru/insurance/responses/company/r...,"""Расчудесная"" история взаимодействия с Сбербан...",NaN,История совершенно свежая. Страховка у меня по...,2016-06-27 13:51:32,1.0,NaN,расчудесный история взаимодействие сбербанк ст...,NaN,NaN,NaN,NaN,NaN
1664,1664,1664,1664,СБС,https://banki.ru/insurance/responses/company/r...,Ужасное отношение,Не засчитана,"Добрый день! 18.02.2016 года, моей маме было н...",2016-03-21 10:44:17,1.0,NaN,ужасный отношение добрый день 18.02 2016 год м...,NaN,NaN,NaN,NaN,NaN


In [121]:
df[(df[type_error] == '') & (df['Оценка'] < 4)][20:40]

,Unnamed: 0,Unnamed: 0.1,Компания,url жалобы,Заголовок,Статус,Текст,Время,Оценка,Оценка выплат,preprocess_Текст,Вид_СС,Тип обращения,Вид_продукта,Номер_полиса,номер обращения CRM
57,57,57,СБСЖ,https://banki.ru/insurance/responses/company/r...,Затягивание в рассмотрении страхового случая,NaN,06.03.2020 оода мною через отделение Сбербанка...,2020-03-16 18:01:03,1.0,1.0,затягивание рассмотрение страховой случай 06.0...,None,,None,None,None
65,65,65,СБСЖ,https://banki.ru/insurance/responses/company/r...,Страхование при потребительском кредите,Не засчитана,В оформление потребительского кредита было вкл...,2020-03-06 12:06:38,1.0,1.0,страхование потребительский кредит оформление ...,None,,None,None,None
72,72,72,СБСЖ,https://banki.ru/insurance/responses/company/r...,"Личный кабинет, страховой случай",NaN,Здравствуйте! Номер полиса ФМСР33 № 0003989302...,2020-03-03 10:44:09,1.0,NaN,личный кабинет страховой случай здравствовать ...,None,,None,'00039893021,None
80,80,80,СБСЖ,https://banki.ru/insurance/responses/company/r...,Невыплата доп инвестиционного дохода,Проблема решена,В январе-20г. были поданы два заявления на вып...,2020-02-21 12:44:20,2.0,NaN,невыплата допы инвестиционный доход январь 20г...,None,,смарт,None,None
85,85,85,СБСЖ,https://banki.ru/insurance/responses/company/r...,Вопросы о ДИДе,NaN,Договор ВМСР50 №000329843 от19.07.18. В течени...,2020-02-18 18:48:24,1.0,NaN,вопрос дидь договор вмср50 № 000329843 19.07 1...,None,,None,None,None
88,88,88,СБСЖ,https://banki.ru/insurance/responses/company/r...,Скрывают номер договора страхования жизни и по...,NaN,Здравствуйте. Был оформлен кредит в сбербанке ...,2020-02-16 19:24:46,1.0,NaN,скрывать номер договор страхование жизнь полис...,None,,None,None,None
89,89,89,СБСЖ,https://banki.ru/insurance/responses/company/r...,Скрытая страховка,NaN,Сбербанк скрывает информацию по договору страх...,2020-02-14 09:21:31,1.0,NaN,скрытый страховка сбербанк скрывать информация...,None,,None,None,200210-0739-753800
90,90,90,СБСЖ,https://banki.ru/insurance/responses/company/r...,Постоянно меняется список документов,Не засчитана,Год назад пенсионерка оформила СТРАХОВАНИЕ ЖИЗ...,2020-02-14 08:54:25,1.0,NaN,постоянно меняться список документ год назад п...,None,,None,None,None
100,100,100,СБСЖ,https://banki.ru/insurance/responses/company/r...,Страхование жизни Билет в будущее,Не засчитана,Здравствуйте!По моему обращению не было ответа...,2020-02-07 15:01:17,1.0,NaN,страхование жизнь билет будущее здравствовать ...,None,,None,None,None
102,102,102,СБСЖ,https://banki.ru/insurance/responses/company/r...,Не возвращают деньги умершей мамы,Не засчитана,Не отпускайте одних пожилых и престарелых родс...,2020-02-06 21:26:28,1.0,1.0,возвращать деньги умирать мама отпускать пожил...,смерть,,смарт,None,None


In [55]:
df.loc[27, 'Текст']

'Всем доброго времени суток. Хочу поделиться с Вами, крайне негативным опытом, о страховой компании "СБЕРБАНК СТРАХОВАНИЕ".\xa015.02.2017 года, мы с супругом оформили две программы накопительной\xa0 страхования жизни(Дог. 7000387*** и 7000387***) Условия были такие: внесение по 5000 рублей по каждому договору, в течение пяти лет, с возможностью получения налогового вычета. За 5 лет накапливается 300000,которые можно будет забрать после окончания срока договора. При наступление страхового случая*, эта сумма удваивается. И так как это страховая компания банка, есть ещё один бонус (5% от суммы платежа), при оформление договора, оформляют автоплатеж. Такой расклад по каждому договору.\xa0И, что мы в итоге получили :1. При оформлении договора, сотрудники страховой компании внесли в свою электронную базу с грубейшими ошибками данные по договору моего мужа, с его паспортными данными, они внесли МОИ ФИО. (Представьте кто там работает? Какой над ними контроль?) Читатель выбирайте сами, что Вас 

In [54]:
df.loc[(df['Текст'].str.contains("ошибка"))]

,Компания,url жалобы,Заголовок,Статус,Текст,Время,Оценка,Оценка выплат,preprocess_Текст,Вид_СС,Тип обращения,Вид_продукта,Номер_полиса,номер обращения CRM
27,СБСЖ,https://banki.ru/insurance/responses/company/r...,Ужасная страховая компания,Не засчитана,Всем доброго времени суток. Хочу поделиться с ...,2020-04-06 15:18:27,1.0,NaN,ужасный страховой компания добрый время сутки ...,None,расторжение | ошибка |,None,None,None
37,СБСЖ,https://banki.ru/insurance/responses/company/r...,Ошибка при страховании имущества,NaN,Добрый день! 17.02.20 мы оформили ипотеку (кре...,2020-03-26 13:05:03,2.0,NaN,ошибка страхование имущество добрый день 17.02...,None,ошибка |,None,None,None
47,СБСЖ,https://banki.ru/insurance/responses/company/r...,Не страхуйтесь там никогда!!!,NaN,Я не любитель писать отзывы. Это очень надо по...,2020-03-20 11:05:09,1.0,NaN,страховаться !!! любитель писать отзыв это оче...,None,ошибка |,None,None,None
123,СБСЖ,https://banki.ru/insurance/responses/company/r...,Корпоративный обман Сбербанк страхование жизни,NaN,Являюсь клиентом Сбербанка Премьер. С 2016 год...,2020-01-24 09:45:31,1.0,1.0,корпоративный обман сбербанк страхование жизнь...,None,ошибка |,None,None,None
250,СБСЖ,https://banki.ru/insurance/responses/company/r...,Нарушение законодательства в части отказа от с...,Проблема решена,У Сбербанка есть хороший пакет Сбербанк премье...,2019-10-11 07:44:26,1.0,NaN,нарушение законодательство часть отказ страхов...,None,ошибка | возврат средств |,семейный актив,None,None
308,СБСЖ,https://banki.ru/insurance/responses/company/r...,Не возвращают деньги,NaN,"Добрый день, меня зовут Дмитрий, 11.07.2019 бр...",2019-08-27 19:00:52,1.0,NaN,возвращать деньги добрый день звать дмитрий 11...,None,расторжение | ошибка | возврат средств |,None,None,None
318,СБСЖ,https://banki.ru/insurance/responses/company/r...,Четвертый месяц жду выплаты после расторжения ...,Проблема решена,Добрый день. Моя история несильно отличается о...,2019-08-21 02:35:00,1.0,NaN,четвертый месяц ждать выплата расторжение дого...,None,расторжение | ошибка | возврат средств |,None,None,None
422,СБСЖ,https://banki.ru/insurance/responses/company/r...,Сберегательное страхование. Как красиво попаст...,NaN,Очаровательная история.Придя как то в сбер под...,2019-05-28 09:32:16,1.0,NaN,сберегательный страхование красиво попадать де...,None,ошибка | возврат средств |,None,'7000469494,None
497,СБСЖ,https://banki.ru/insurance/responses/company/r...,Где моя выкупная сумма,NaN,Добрый день.22 февраля я подала заявление на р...,2019-04-12 17:53:08,1.0,2.0,выкупной сумма добрый день 22 февраль подавать...,None,расторжение | ошибка |,None,None,None
498,СБСЖ,https://banki.ru/insurance/responses/company/r...,"Центральный Банк, примите меры!",NaN,С 22 марта не могу подать заявление о расторже...,2019-04-12 09:19:29,1.0,NaN,центральный банк принимать мера 22 март мочь п...,None,расторжение | ошибка |,None,None,None


In [21]:
df.loc[193, 'Текст']

'24.07.2019 умерла моя мама. У неё был кредит в сбербанке почти на 300 000 рублей, застрахованный по программе страхование жизни от сбербанка. 6.08.19 были отправлены документы в страховую компанию через отделение сбербанка в г. Воронеж по адресу Кольцовская 46. Далее пришло письмо от страховой компании о том что нужно дослать выписку из амбулаторной карты и оригинал договора. Так как оригинала договора у меня не было его пришлось запросить в архиве. И ВНИМАНИЕ!!! Если у вас нет оригинала договора со страховой то вам придется его запросить в архиве страховой компании через отделение сбербанка, что бы страховая отправила оригинал из архива в отделение сбербанка, а потом сбербанк отправил этот договор в страховую обратно. Спустя месяц пришла выписка из амбулаторной карты из одной из поликлиник. А из второй поликлиники ответ не пришел. Я позвонила на горячую линию там сообщили что ответа от второй поликлиники не получали. Мы сходили в поликлинику и главврач сообщил что никаких запросов от

In [50]:
df.loc[df['preprocess_Текст'].str.contains('ошибка'), 'Текст']

27      Всем доброго времени суток. Хочу поделиться с ...
37      Добрый день! 17.02.20 мы оформили ипотеку (кре...
47      Я не любитель писать отзывы. Это очень надо по...
62      Заключение договора с ООО СК «Сбербанк страхов...
67      05.03.2020 продлил страхование жизни по своей ...
                              ...                        
1598    В апреле 2016 года оформила ипотеку. Сразу сде...
1603    Сразу хочу сказать, что зарегистрироваться на ...
1617    Добрый день.Являясь заемщиком Сбербанка по ипп...
1648    12 октября 2016 г. заключила договор о страхов...
1663    После январских праздников, Сбербанк сообщил м...
Name: Текст, Length: 98, dtype: object